In [1]:
ls

crosstalk_thoughts.ipynb                 performance_of_correction.png
crosstalk_vs_rate.jpg                    plots_from_jens/
foo.png                                  poisson_correction.ipynb
foo.svg                                  README.md
formula_foo.png                          single_pe_spectra.hdf
gedankenexperiment_with_crosstalk.ipynb  Untitled2.ipynb
papers/


In [2]:
input_file_path = './single_pe_spectra.hdf'

In [3]:
import h5py

/home/dneise/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [4]:
df = pd.read_hdf(input_file_path, key='Baseline')
df.head()

,,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
night,run_id,,,,,,,,,,,,,,,,,,,,,
2017-12-23,0,0.029269,-0.668304,0.255208,0.471072,0.277753,0.011224,0.34222,0.214303,0.32629,-0.357545,...,0.289704,0.310159,0.023236,-0.183069,-0.353909,-0.093427,-0.028284,-0.189777,-0.308397,0.0131


In [5]:
R = pd.read_hdf(input_file_path, key='Rate')

In [7]:
with h5py.File(input_file_path) as f:
    keys = list(f.keys())
    print(*keys, sep=" | ")

Baseline | Chi2 | CoeffR | Crosstalk | FitProb | Gain | Noise | NormGain | Pxtalk | Rate | RelSigma | Signal


In [8]:
t_0 = 7e-9

In [9]:
mu = R.iloc[0][0] * 1e6 * t_0

In [10]:
mu = R * 1e6 * t_0

In [11]:
from scipy.stats import poisson
from scipy.special import factorial

In [12]:
Nmax = 20

In [13]:
P_mu = poisson.pmf(k=np.arange(0, Nmax)[:, None], mu=mu)

In [23]:
p = pd.read_hdf(input_file_path, key='Pxtalk')
nu = pd.read_hdf(input_file_path, key='CoeffR')
p_xt = pd.read_hdf(input_file_path, key='Crosstalk')
q = p * np.exp(-p)

In [15]:
def calc_Pn(N, q, nu):
    return (q*N)**(N-1) / (factorial(N-1))**nu

In [16]:
Pn = np.zeros((Nmax, 1440))
for n in range(1, Nmax+1):
    Pn[n-1] = calc_Pn(n , q, nu)

In [17]:
Pn.shape, P_mu.shape

((20, 1440), (20, 1440))

In [19]:
Pn /= Pn.sum(axis=0)

In [22]:
Foo = Pn - P_mu

In [25]:
p_xt

,,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
night,run_id,,,,,,,,,,,,,,,,,,,,,
2017-12-23,0,0.146648,0.150309,0.13104,0.130837,0.146146,0.136665,0.16946,0.172331,0.140599,0.122513,...,0.132108,0.13999,0.166443,0.127064,0.160327,0.168607,0.144601,0.172661,0.153444,0.154155


In [27]:
Pn[1:].sum(axis=0)

array([0.14664759, 0.15030914, 0.13103951, ..., 0.17266154, 0.15344385,
       0.15415537])